### portmy database: stocks table
### stock database: buy, price tables

In [1]:
import pandas as pd
import numpy as np
from datetime import date, timedelta
from sqlalchemy import create_engine

engine = create_engine("sqlite:///c:\\ruby\\portmy\\db\\development.sqlite3")
conmy = engine.connect()
conmy = engine.connect()
engine = create_engine("mysql+pymysql://root:@localhost:3306/stock")
const = engine.connect()

pd.set_option('display.float_format','{:,.2f}'.format)

data_path = "../data/"
csv_path = "\\Users\\User\\iCloudDrive\\"
box_path = "\\Users\\User\\Dropbox\\"

today = date.today()
week_str = today.strftime('%Y-%U')
today, week_str

(datetime.date(2022, 4, 30), '2022-17')

### Process after last business day, today must be last business day

In [2]:
today = today - timedelta(days=1)
wkago = today - timedelta(days=10)
today, wkago

(datetime.date(2022, 4, 29), datetime.date(2022, 4, 19))

### Restart & Run All Cells

### This process affects only already in port stocks. To highlight price changes in percent.

In [3]:
cols = 'name price_d price_w percent perform mrkt '.split()

In [4]:
format_dict = {
    'qty':'{:,}',    
    'price':'{:.2f}','price_d':'{:.2f}','price_w':'{:.2f}',
    'max_price':'{:.2f}','min_price':'{:.2f}',
    'maxp':'{:.2f}','minp':'{:.2f}','opnp':'{:.2f}',    
    'pct':'{:,.2f}%','percent':'{:,.2f}%',
    
    'pe':'{:.2f}','pbv':'{:.2f}',
    'paid_up':'{:,.2f}','market_cap':'{:,.2f}',
    'daily_volume':'{:,.2f}','beta':'{:,.2f}', 
    'dly_vol':'{:,.2f}', 
}

In [5]:
sql = """
SELECT name, date, price 
FROM buy 
ORDER BY date DESC 
LIMIT 1
"""
df_buy = pd.read_sql(sql, const)
df_buy.style.format(format_dict)

,name,date,price
0,ORI,2022-04-07,11.00


In [6]:
names = df_buy['name']
name = names.to_string(index=False)

sql = '''
SELECT * FROM stocks WHERE name = '%s'
'''
sql = sql % name
print(sql)

df_stocks = pd.read_sql(sql, conmy)
df_stocks.style.format(format_dict) 


SELECT * FROM stocks WHERE name = 'ORI'



,id,name,market,price,max_price,min_price,pe,pbv,paid_up,market_cap,daily_volume,beta,ticker_id,created_at,updated_at
0,339,ORI,SET100 / SETHD / SETTHSI,11.10,12.70,8.05,8.53,1.82,"1,227.06","27,240.76",116.27,1.32,339,2017-07-23 07:25:07.542864,2022-04-23 07:59:17.253854


In [7]:
sql = '''
SELECT * FROM price WHERE name = "%s" ORDER BY date DESC LIMIT 5
'''
sql = sql % name
print(sql)

df_price = pd.read_sql(sql, const)
df_price.style.format(format_dict)


SELECT * FROM price WHERE name = "ORI" ORDER BY date DESC LIMIT 5



,name,date,price,maxp,minp,qty,opnp
0,ORI,2022-04-29,11.00,11.20,10.90,"4,964,172",11.10
1,ORI,2022-04-28,11.10,11.10,10.90,"2,344,788",10.90
2,ORI,2022-04-27,10.90,11.10,10.80,"3,238,896",11.00
3,ORI,2022-04-26,11.00,11.20,11.00,"2,544,792",11.10
4,ORI,2022-04-25,11.00,11.10,10.90,"3,331,566",11.00


In [8]:
sql = '''
SELECT name
FROM buy 
WHERE active = 1
ORDER BY name'''
df_price = pd.read_sql(sql, const)

names = df_price.name.tolist()
in_p = ", ".join(map(lambda name: "'%s'" % name, names))

sql = """
SELECT name, price AS price_d 
FROM price 
WHERE date = '%s' AND name IN (%s)
ORDER BY name, date"""
sql = sql % (today, in_p)
print(sql)

df_today = pd.read_sql(sql, const)
df_today.shape[0]


SELECT name, price AS price_d 
FROM price 
WHERE date = '2022-04-29' AND name IN ('ASP', 'BCH', 'DIF', 'DOHOME', 'HREIT', 'IP', 'JASIF', 'KCE', 'MAKRO', 'MCS', 'NER', 'ORI', 'PTT', 'PTTGC', 'RATCH', 'RCL', 'RJH', 'SCC', 'SCCC', 'SENA', 'SPALI', 'STA', 'SYNEX', 'TISCO', 'TMT', 'TPIPL', 'WHART')
ORDER BY name, date


27

In [9]:
sql = """
SELECT name, price AS price_w
FROM price 
WHERE date = '%s' AND name IN (%s) 
ORDER BY name"""
sql = sql % (wkago, in_p)
print(sql)

df_wkago = pd.read_sql(sql, const)
df_wkago.shape[0]


SELECT name, price AS price_w
FROM price 
WHERE date = '2022-04-19' AND name IN ('ASP', 'BCH', 'DIF', 'DOHOME', 'HREIT', 'IP', 'JASIF', 'KCE', 'MAKRO', 'MCS', 'NER', 'ORI', 'PTT', 'PTTGC', 'RATCH', 'RCL', 'RJH', 'SCC', 'SCCC', 'SENA', 'SPALI', 'STA', 'SYNEX', 'TISCO', 'TMT', 'TPIPL', 'WHART') 
ORDER BY name


27

In [10]:
trend = pd.merge(df_today, df_wkago, on='name',how='inner')
trend.shape

(27, 3)

In [11]:
def performance(vals):
    price_d, price_w = vals
    if (price_d > price_w):
        return 'Better'
    elif (price_d < price_w):
        return 'Worse'
    else:
        return 'No Change'

In [12]:
trend['percent'] = (trend.price_d-trend.price_w)/trend.price_w * 100

In [13]:
trend

,name,price_d,price_w,percent
0,ASP,3.38,3.40,-0.59
1,BCH,21.50,22.40,-4.02
2,DIF,14.10,14.20,-0.70
3,DOHOME,20.10,22.10,-9.05
4,HREIT,8.50,8.60,-1.16
5,IP,19.60,20.10,-2.49
6,JASIF,11.00,11.00,0.00
7,KCE,61.50,61.25,0.41
8,MAKRO,37.50,39.00,-3.85
9,MCS,12.30,12.80,-3.91


In [14]:
trend["perform"] = trend[["price_d","price_w"]].apply(performance, axis=1)

In [15]:
trend.sort_values(by=['percent'],ascending=[True]).style.format(format_dict)

,name,price_d,price_w,percent,perform
23,TISCO,90.75,100.00,-9.25%,Worse
3,DOHOME,20.10,22.10,-9.05%,Worse
1,BCH,21.50,22.40,-4.02%,Worse
9,MCS,12.30,12.80,-3.91%,Worse
25,TPIPL,1.50,1.56,-3.85%,Worse
8,MAKRO,37.50,39.00,-3.85%,Worse
21,STA,25.25,26.25,-3.81%,Worse
26,WHART,10.90,11.30,-3.54%,Worse
22,SYNEX,24.20,25.00,-3.20%,Worse
24,TMT,10.10,10.40,-2.88%,Worse


In [16]:
trend.perform.value_counts(normalize=True).to_frame().style.format('{:.2%}')

,perform
Worse,74.07%
Better,18.52%
No Change,7.41%


In [17]:
sql = '''
SELECT name, max_price AS max, min_price AS min, pe, pbv, daily_volume AS volume, beta, market
FROM stocks
'''
my_stocks = pd.read_sql(sql, conmy)
my_stocks.shape

(359, 8)

In [18]:
filters = [
   (my_stocks.market.str.contains('SET50')),
   (my_stocks.market.str.contains('SET100')),
   (my_stocks.market.str.contains('mai'))]
values = ['SET50','SET100','mai']
my_stocks["mrkt"] = np.select(filters, values, default='SET999')

In [19]:
trend2 = pd.merge(trend, my_stocks, on='name', how='inner')
trend2.sort_values(['percent'],ascending=[True]).shape

(27, 13)

In [20]:
set50 = trend2.mrkt.str.contains('SET50') 
flt_set50 = trend2[set50]
flt_set50[cols].sort_values(by=['percent','name'],ascending=[True,True]).style.format(format_dict)

,name,price_d,price_w,percent,perform,mrkt
23,TISCO,90.75,100.00,-9.25%,Worse,SET50
14,RATCH,44.00,44.75,-1.68%,Worse,SET50
12,PTT,37.50,37.75,-0.66%,Worse,SET50
7,KCE,61.50,61.25,0.41%,Better,SET50
13,PTTGC,50.00,49.50,1.01%,Better,SET50
17,SCC,370.00,360.00,2.78%,Better,SET50


In [21]:
flt_set50.perform.value_counts(normalize=True).to_frame().style.format('{:.2%}')

,perform
Better,50.00%
Worse,50.00%


In [22]:
set100 = trend2.mrkt.str.contains('SET100')
flt_set100 = trend2[set100]
flt_set100[cols].sort_values(by=['percent','name'],ascending=[True,True]).style.format(format_dict)

,name,price_d,price_w,percent,perform,mrkt
3,DOHOME,20.10,22.10,-9.05%,Worse,SET100
1,BCH,21.50,22.40,-4.02%,Worse,SET100
21,STA,25.25,26.25,-3.81%,Worse,SET100
22,SYNEX,24.20,25.00,-3.20%,Worse,SET100
20,SPALI,21.90,22.30,-1.79%,Worse,SET100
11,ORI,11.00,11.20,-1.79%,Worse,SET100
15,RCL,44.75,44.50,0.56%,Better,SET100


In [23]:
flt_set100[cols].perform.value_counts(normalize=True).to_frame().style.format('{:.2%}')

,perform
Worse,85.71%
Better,14.29%


In [24]:
set999 = trend2.mrkt.str.contains('SET999')
flt_set999 = trend2[set999]
flt_set999[cols].sort_values(by=['percent','name'],ascending=[True,True]).style.format(format_dict)

,name,price_d,price_w,percent,perform,mrkt
9,MCS,12.30,12.80,-3.91%,Worse,SET999
25,TPIPL,1.50,1.56,-3.85%,Worse,SET999
8,MAKRO,37.50,39.00,-3.85%,Worse,SET999
26,WHART,10.90,11.30,-3.54%,Worse,SET999
24,TMT,10.10,10.40,-2.88%,Worse,SET999
10,NER,6.75,6.90,-2.17%,Worse,SET999
4,HREIT,8.50,8.60,-1.16%,Worse,SET999
2,DIF,14.10,14.20,-0.70%,Worse,SET999
18,SCCC,152.00,153.00,-0.65%,Worse,SET999
0,ASP,3.38,3.40,-0.59%,Worse,SET999


In [25]:
flt_set999[cols].perform.value_counts(normalize=True).to_frame().style.format('{:.2%}')

,perform
Worse,76.92%
No Change,15.38%
Better,7.69%


In [26]:
setmai = trend2.mrkt.str.contains('mai')
flt_setmai = trend2[setmai]
flt_setmai[cols].sort_values(by=['percent','name'],ascending=[True,True]).style.format(format_dict)

,name,price_d,price_w,percent,perform,mrkt
5,IP,19.60,20.10,-2.49%,Worse,mai


In [27]:
flt_setmai[cols].perform.value_counts(normalize=True).to_frame().style.format('{:.2%}')

,perform
Worse,100.00%
